# Experimenting with brute force engineering with most important variables

## Most important variables were:

#### Logistic Regression: V14, V12, V4, V17, V10, V8, V11, V16, V3, V1, V7, V18, V25
#### Random Forest: 'V17', 'V10', 'V12', 'V14', 'V16', 'V11', 'V9', 'V4', 'V18', 'V7','V26', 'V20'
#### Random Forest SMOTE: 'V10', 'V14', 'V12', 'V11', 'V17', 'V4', 'V16', 'V7', 'V18', 'V9', V20', 'V26'

In [14]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score
from sklearn.linear_model import LogisticRegression

In [2]:
df = pd.read_csv ('../../data/creditcard_v2.csv')

In [3]:
most_important_variables = ['V10', 'V14', 'V12', 'V17', 'V16', 'V11', 'V7', 'V18', 'V4', 'V9']

In [4]:
df_most_important = df [most_important_variables]

In [5]:
df_features = df_most_important.copy (deep = True)

In [6]:
def ratios (df):
    for i in most_important_variables:
        for j in most_important_variables:
            if i==j:
                continue
            df [i + '/' + j] = df [i].div (df [j])
    return df

In [7]:
df_features = ratios (df_features)

In [8]:
df_features.shape

(283296, 100)

In [9]:
df_features.sample (5)

,V10,V14,V12,V17,V16,V11,V7,V18,V4,V9,...,V4/V9,V9/V10,V9/V14,V9/V12,V9/V17,V9/V16,V9/V11,V9/V7,V9/V18,V9/V4
130887,0.544606,0.355002,0.728548,0.154798,-1.588680,1.144340,0.383128,1.136460,-0.082578,-1.130659,...,0.073035,-2.076103,-3.184934,-1.551935,-7.304110,0.711697,-0.988044,-2.951128,-0.994896,13.692026
262237,0.058024,0.227142,0.567615,-0.271546,-0.155027,-0.711245,0.184288,-0.966287,0.389285,0.507263,...,0.767422,8.742342,2.233245,0.893674,-1.868055,-3.272093,-0.713204,2.752553,-0.524961,1.303064
151093,-1.135690,0.568145,-1.984969,0.935473,-0.335595,1.733924,1.658164,0.811881,-0.849635,0.838119,...,-1.013740,-0.737982,1.475185,-0.422233,0.895931,-2.497410,0.483365,0.505450,1.032317,-0.986446
101638,-0.912187,0.181554,0.521546,-0.860063,1.228582,-1.337824,-0.410091,-0.357822,-2.202545,-0.021457,...,102.648937,0.023523,-0.118185,-0.041141,0.024948,-0.017465,0.016039,0.052323,0.059966,0.009742
62378,-1.332940,0.244007,0.031438,-0.189041,0.135662,-2.067198,-1.043374,-0.553731,-0.511147,0.339016,...,-1.507739,-0.254337,1.389371,10.783654,-1.793345,2.498967,-0.163998,-0.324922,-0.612239,-0.663245


## Normal dataset

In [10]:
X_train, X_test, y_train, y_test = train_test_split (df, df.Class, 
                                                    stratify = df.Class,
                                                    test_size = 0.25,
                                                    random_state = 0)

rf_1 = RandomForestClassifier ()
model = rf_1.fit (X_train, y_train)

print ('Precision: ' + 
       str (precision_score (y_test, model.predict (X_test))) +
       ', Recall: ' +
       str (recall_score (y_test, model.predict (X_test))))

Precision: 1.0, Recall: 1.0


In [18]:
X_train, X_test, y_train, y_test = train_test_split (df, df.Class, 
                                                    stratify = df.Class,
                                                    test_size = 0.25,
                                                    random_state = 0)

lr_1 = LogisticRegression (max_iter=1000)
model = lr_1.fit (X_train, y_train)

print ('Precision: ' + 
       str (precision_score (y_test, model.predict (X_test))) +
       ', Recall: ' +
       str (recall_score (y_test, model.predict (X_test))))

Precision: 0.8681318681318682, Recall: 0.6422764227642277


## Most important

In [12]:
X_train_i, X_test_i, y_train_i, y_test_i = train_test_split (df_most_important, df.Class, 
                                                    stratify = df.Class,
                                                    test_size = 0.25,
                                                    random_state = 0)

rf_2 = RandomForestClassifier ()
model_important = rf_2.fit (X_train_i, y_train_i)

print ('Precision: ' + 
       str (precision_score (y_test_i, model_important.predict (X_test_i))) +
       ', Recall: ' +
       str (recall_score (y_test_i, model_important.predict (X_test_i))))

Precision: 0.912621359223301, Recall: 0.7642276422764228


In [16]:
X_train_i, X_test_i, y_train_i, y_test_i = train_test_split (df_most_important, df.Class, 
                                                    stratify = df.Class,
                                                    test_size = 0.25,
                                                    random_state = 0)

lr_2 = LogisticRegression ()
model_important = lr_2.fit (X_train_i, y_train_i)

print ('Precision: ' + 
       str (precision_score (y_test_i, model_important.predict (X_test_i))) +
       ', Recall: ' +
       str (recall_score (y_test_i, model_important.predict (X_test_i))))

Precision: 0.8928571428571429, Recall: 0.6097560975609756


## Brute force

In [13]:
X_train_b, X_test_b, y_train_b, y_test_b = train_test_split (df_features, df.Class, 
                                                    stratify = df.Class,
                                                    test_size = 0.25,
                                                    random_state = 0)

rf_3 = RandomForestClassifier ()
model_all_features = rf_3.fit (X_train_b, y_train_b)

print ('Precision: ' + 
       str (precision_score (y_test_b, model_all_features.predict (X_test_b))) +
       ', Recall: ' +
       str (recall_score (y_test_b, model_all_features.predict (X_test_b))))

Precision: 0.9108910891089109, Recall: 0.7479674796747967


In [21]:
X_train_b, X_test_b, y_train_b, y_test_b = train_test_split (df_features, df.Class, 
                                                    stratify = df.Class,
                                                    test_size = 0.25,
                                                    random_state = 0)

lr_3 = LogisticRegression (max_iter=1000000, tol=0.00001)
model_all_features = lr_3.fit (X_train_b, y_train_b)

print ('Precision: ' + 
       str (precision_score (y_test_b, model_all_features.predict (X_test_b))) +
       ', Recall: ' +
       str (recall_score (y_test_b, model_all_features.predict (X_test_b))))

Precision: 0.8351648351648352, Recall: 0.6178861788617886


### Exploring variable importance

In [28]:
variable_coefficient = (
    pd.DataFrame (
        [df_most_important.columns, abs (model_important.feature_importances_)])
    .transpose ()
    .sort_values (by = 1, ascending = False))

In [29]:
variable_coefficient.head (20).reset_index(drop = True)

,0,1
0,V17,0.222552
1,V12,0.198274
2,V14,0.148295
3,V10,0.130022
4,V16,0.066594
5,V11,0.064038
6,V4,0.0466989
7,V7,0.0442519
8,V9,0.0423089
9,V18,0.0369653


In [ ]:
df_features.hist (figsize = (16,20), bins = 50, xlabelsize=8, ylabelsize=8);